# Import des constantes

In [8]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

#Tokenization 
import nltk
from gensim.models.phrases import Phrases, Phraser
from nltk.tokenize import TweetTokenizer
from functools import reduce
from operator import add
import collections

#Word2Vec
from gensim.models import Word2Vec
from multiprocessing import cpu_count

#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#BERT
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

#Clustering 
from sklearn.cluster import KMeans

# Import de la base

In [9]:
# Import de la base intiale 
f = open('1377884570_tweet_global_warming.txt', 'r',newline='', encoding='ISO-8859-1')
content = f.read().split('\r')

content_new=[]
for x in content : 
    if len(x)>0:
        content_new.append(x)

content_new=content_new[1:len(content_new)]

In [10]:
# Création du dataframe

col_tweet=[]
col_existence=[]
col_score=[]

#Split tweet , Note

for line in content_new:
    if len(line.split('[link]'))==2:
        (x,y)=line.split('[link]')
        col_tweet.append(x)
        col_existence.append(y)
    else : 
        if len(line.split(',Yes,'))==2:
            col_tweet.append(line.split(',Yes,')[0])
            col_existence.append(',Yes,'+line.split(',Yes,')[1])
        elif len(line.split(',No,'))==2:
            col_tweet.append(line.split(',No,')[0])
            col_existence.append(',No,'+line.split(',No,')[1])
        elif len(line.split(',Y,'))==2:
            col_tweet.append(line.split(',Y,')[0])
            col_existence.append(',Yes,'+line.split(',Y,')[1])
        elif len(line.split(',N/A,'))==2:
            col_tweet.append(line.split(',N/A,')[0])
            col_existence.append(',N/A,'+line.split(',N/A,')[1])
        elif len(line.split(',NA,'))==2:
            col_tweet.append(line.split(',NA,')[0])
            col_existence.append(',NA,'+line.split(',NA,')[1])
        elif len(line.split(',N,'))==2:
            col_tweet.append(line.split(',N,')[0])
            col_existence.append(',No,'+line.split(',N,')[1])
        else : 
            print('erreur')
            #print(line.split('[link]'))
col_tweet.append('I truly  Fat ASS Gore should get the Scam Artist Award of the decade with his Global Warming and Energy Credits worth close to Billion')
col_existence.append(' ,NA')
col_tweet.append('Despite Climategate, LEFT investing heavily in global warming hysteria as new way 2 impose nat\'l & international controls on human freedom.')
col_existence.append(' ,NA')
        
# Split Existence/Note
col_existence_new=[]

for x in col_existence:
    if len(x.split(','))==3:
        col_existence_new.append(x.split(',')[1])
        col_score.append(x.split(',')[2])
    else:
        col_existence_new.append('NA')
        col_score.append('NA')
        
#Nettoyage existence
for avis in range(len(col_existence_new)):
    if col_existence_new[avis]=='NA' or col_existence_new[avis]=='N/A' or col_existence_new[avis]=='':
        col_existence_new[avis]=np.nan
        
#Nettoyage score
for score in range(len(col_score)):
    if 'NA' not in col_score[score]:
        col_score[score]=col_score[score].split('\t')[0]
        if len(col_score[score].split('"'))>1:
            col_score[score]=float(col_score[score].split('"')[0])
        else: 
            col_score[score]=float(col_score[score])
            
    else : 
        col_score[score]=np.nan

#Creation du DataFrame
dic={'Tweet':col_tweet,'Existence':col_existence_new,'Score':col_score}
df=pd.DataFrame(dic)
    

erreur
erreur


In [11]:
df.drop_duplicates(['Tweet'], inplace=True)
df.reset_index(inplace=True,drop=True)

# Quelques statistiques

In [ ]:
# Exemples de tweets

print('Exemples de tweets qui croient au réchauffement climatique : ')
print(' ')
for k in range(5):
    print(df[df.Existence=='Yes'].reset_index().iloc[k]['Tweet'])
    print(' ')
    
print('#'*50)
print('#'*50)
print(' ')
print('Exemples de tweets qui doutent du réchauffement climatique : ')
print(' ')
for k in range(5):
    print(df[df.Existence=='No'].reset_index().iloc[k]['Tweet'])
    print(' ')

In [ ]:
print('On dispose de {} tweets.'.format(df.shape[0]))

print(' ')
print('On a {} données manquantes sur le label de l\'avis du tweet (Yes, No) .'.format(str(df.isnull().sum()['Existence'])))
print(' ')

#Personnes convaincues du changement climatique 
print('On a {} tweets qui croit au Changement climatique.'.format(str(df[df.Existence=='Yes'].shape[0])))
m=round(df[df.Existence=='Yes'].Score.describe()['mean'],2)
std=round(df[df.Existence=='Yes'].Score.describe()['std'],2)
print('Parmis ces personnes, le score est de {} en moyenne, avec un écart-type de {}.'.format(str(m),str(std)))
print(' ')

#Personnes qui doutent du changement climatique 
print('On a {} tweets qui remettent en doute le Changement climatique.'.format(str(df[df.Existence=='No'].shape[0])))
m=round(df[df.Existence=='No'].Score.describe()['mean'],2)
std=round(df[df.Existence=='No'].Score.describe()['std'],2)
print('Parmis ces personnes, le score est de {} en moyenne, avec un écart-type de {}.'.format(str(m),str(std)))


In [ ]:
x=plt.hist(list(df.Score))
x=plt.title('Histogramme des scores de la base globale')

In [ ]:
x=plt.hist(list(df[df.Existence=='Yes'].Score))
x=plt.title('Histogramme des scores pour les tweets qui croient au réchauffement climatique ')

In [ ]:
x=plt.hist(list(df[df.Existence=='No'].Score))
x=plt.title('Histogramme des scores pour les tweets qui ne croient pas au réchauffement climatique ')

Les scores sont des indicateurs sur la precision de la classification du tweet. Globalement, on a la même certitude sur cette précision pour les deux types de tweets (Yes et No).

# Tokenization des tweets

In [12]:
#Nombre de mots différents dans l'ensemble des articles 

##Récupération de tout les mots de tout les tweets
arr=df.Tweet.apply(lambda x : x.split(' ')).array
arr = reduce(add, arr)


##Nombre de mots différents dans l'ensemble des articles 
print('En utilisant le séparateur \' \', on obtient {} mots dans l\'ensemble des tweets'.format(len(set(arr))))


En utilisant le séparateur ' ', on obtient 19602 mots dans l'ensemble des tweets


In [13]:
# Tokenization en utilisant le Tokenizer Tweeter

arr_tokens = df.Tweet.apply(lambda x: TweetTokenizer().tokenize(x)).array
arr_tokens = reduce(add, arr_tokens)
print('En utilisant le Tokenizer TweetTokenizer, on obtient un vocabulaire de {} mots.'.format(len(set(arr_tokens))))

En utilisant le Tokenizer TweetTokenizer, on obtient un vocabulaire de 15804 mots.


# Nettoyage des Tokens

In [14]:
#On prend toutes les phrases de touts les texts, et on les concatène dans une liste, en les traitant auparavant

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

def remove_hashtags(tokens):
    tokens= map(lambda x : x.replace('#',''),tokens) #map : parcours tout les tokens
    return list(tokens)

def remove_url(tokens): #pb pour https
    tokens= filter(lambda x: "http" not in x, tokens) #filter : garde là où il y a True
    return list(tokens)

def remove_html(tokens):
    tokens= filter(lambda x: x[0]+x[-1]!='<>',tokens)
    return list(tokens)

def remove_www(tokens):
    tokens= filter(lambda x: "www" not in x, tokens) #filter : garde là où il y a True
    return list(tokens)

'''
def remove_x95(clean_corpus):
    for sentence_r in range(len(clean_corpus)):
        sentence=clean_corpus[sentence_r]
        for x in range(len(sentence)):
            if '\x95' in sentence[x]:
                y=sentence[x].split('\x95')
                new_x=''
                for part_x in y:
                    new_x=new_x+part_x
                sentence[x]=new_x
        clean_corpus[sentence_r]=sentence
    return(clean_corpus)
'''

def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def clean_ponctuation(text_tokens): # Nettoyage de la ponctuation

    list_word_clean_ponctuation=[]
    for tweet in text_tokens:
        list_tweet=[]
        for word in tweet:
            if len(word)<2:
                if (word=='a') or (word=='i') or (word=='u'):
                    list_tweet.append(word)
                if RepresentsInt(word):
                    list_tweet.append(word)
            else :
                if (word!='..') & (word!='...') & (word!='rt'):
                    list_tweet.append(word)
        list_word_clean_ponctuation.append(list_tweet)
    
    return(list_word_clean_ponctuation)

def remove_arobase(text_tokens):
    
    list_new_tokens=[]
    for tweet in text_tokens:
        new_tweet=[]
        for word in tweet: 
            if '@' not in word:
                new_tweet.append(word)
        list_new_tokens.append(new_tweet)
    
    return(list_new_tokens)

###############################################################
###############################################################
###############################################################

def clean_text_first(corpus):
    
    tok=TweetTokenizer()
    tokens=[]
    for sample in corpus:
        token=tok.tokenize(sample) 
        token=remove_url(token)
        token=remove_html(token)
        token=remove_hashtags(token)
        token=remove_www(token)
        token=list(map(lambda x : x.lower(),token)) #.lower() : met les majuscules en minuscules
        tokens.append(token) #ajout du token à l'ensemble des phrases
    
    #Nettoyage de la ponctuation
    tokens=clean_ponctuation(tokens)
    
    #Nettoyage des arobase : pour la plupart, se sont des noms propres
    tokens=remove_arobase(tokens)
    
    return tokens

def clean_text_second(corpus,threshold): #On rajoute l'association des mots qui vont ensembles
    
    #clean les textes
    tokens=clean_text_first(corpus)
    
    #associer les mots
    phrases=Phrases(tokens,threshold=threshold) #On fait apprendre le modèle d'association sur tout les mots
    phraser=Phraser(phrases) #Outil pour associer
    
    clean_tokens=[]
    for token in tokens: #On parcours les phrases et on associe les mots
        new_tokens=phraser[token]
        clean_tokens.append(new_tokens)
        
    #tokens = remove_x95(tokens)
    
    return(clean_tokens)


In [15]:
#clean_text_first(df.Tweet) #sans association de mots

In [16]:
list_words=clean_text_second(df.Tweet,threshold=1000) #avec association de mots

In [17]:
#Mots les plus fréquents après nettoyage des tokens

counter=collections.Counter(reduce(add, list_words))

#10 mots les plus fréquents 
number_word=10
print('Les {} mots qui apparaissent le plus sont (par ordre décroissant) :'.format(number_word))
print(' ')
for word in counter.most_common(number_word):
    print(word[0]+' ('+str(word[1])+') ') 

Les 10 mots qui apparaissent le plus sont (par ordre décroissant) :
 
climate (3348) 
change (3027) 
global (2869) 
warming (2765) 
the (2275) 
to (1674) 
of (1440) 
on (1063) 
a (1056) 
in (998) 


In [18]:
# Taille moyenne des tweet : 
list_words
mean=0
for tweet in list_words:
    mean=mean+len(tweet)
    
print('En moyenne, on a retenu {} mots par tweet '.format(str(round(mean/len(list_words),0))))

En moyenne, on a retenu 14.0 mots par tweet 


# Représentation des Tweets

## Avec Word2Vec

On utilise tout d'abord l'algorithme Word2Vec pour représenter ces tweets. Chaque mot à une représentation vectorielle. Pour chaque tweet, on fait la moyenne des vecteurs (chaque mot) inclut dans ce tweet.

In [ ]:
#Cleaning des tweets
clean_text=clean_text_second(df.Tweet,threshold=1000)

print("Entrainement du modèle Word2Vec ...")
model = Word2Vec(clean_text, size=100, window=5, min_count=3, workers=4) 

model.train(clean_text, total_examples=len(clean_text), epochs=10) #réseau de neuronne du Word2Vec
model_wv = model.wv

In [ ]:
# Représentation des tweets en moyennant les mots

def tokens2vectors(tokenCorpus):
    ''' transforms our X into a list of list of vec (2D array) '''
    new_sample = list()
    i=0
    for sample in tokenCorpus:
        tweetVecs = list()
        for token in sample:
            try : 
                tweetVecs.append(model_wv.get_vector(token)  )
            except: 
                i=i+1
                tweetVecs.append( np.zeros(100) ) 
        new_sample.append(np.mean(tweetVecs, axis=0))
    
    return np.array(new_sample)


X= tokens2vectors(clean_text)

Y=[]
for x in list(X):
    try: Y.append(list(x))
    except : pass
    
df_representation_W2v= pd.DataFrame(Y).T

In [ ]:
df_representation_W2v

In [ ]:
# Etude de cette représentation via la cosinus similarité

def closest_to(word,n_top_similar):
    print('Les {} mots les plus proches de {} sont : '.format(n_top_similar,word))
    print(' ')
    for word in [w[0] for w in model_wv.most_similar(word,topn=n_top_similar)]:
        print(word)
        
closest_to('change',10)
print(' ')
print('#'*50)
print(' ')

closest_to('climate',10)

# Une approche naïve avec TF IDF


Pour chaque mot, on a un vecteur le représentant. Pour réprésenter un tweet, on fait la moyenne des vecteurs correspondant à chacun de ces mots.

##### Représentation des mots

In [ ]:
# Initialisation des mots TF-IDF
cv=CountVectorizer()
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)

In [ ]:
# Formatage de la base pour la méthode TF-IDF
corpus=clean_text_second(df.Tweet,threshold=2000)
corpus_new=[]
for tweet in corpus: 
    tweet_sentence=''
    for word in tweet:
        tweet_sentence=tweet_sentence+' '+word
    corpus_new.append(tweet_sentence)
    
# Caclul des scores TF-IDF de chaque mot
word_count_vector=cv.fit_transform(corpus_new)
tfidf_transformer.fit(word_count_vector)

count_vector=cv.transform(corpus_new)
tf_idf_vector=tfidf_transformer.transform(count_vector)
feature_names = cv.get_feature_names()


In [ ]:
# Exemple de représentation du premier tweet 

#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]

#print du tweet
print(' ')
print('Tweet initial : '+df.iloc[0]['Tweet'])
print(' ')

#print du tweet nettoyé
print('Tweet nettoyé : '+corpus_new[0])
print(' ')
print('Scores TF-IDF : ')
#print des scores 
pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["first_tweet"]).sort_values(by=["first_tweet"],ascending=False)

In [ ]:
# Représentation de tout les tweets 

dt_tfidf_tweet = pd.DataFrame(tf_idf_vector[0:2].T.todense(),index=feature_names)
dt_tfidf_tweet

Problématique : matrice très sparse (beaucoup de zéros)

# Avec du clustering de mots d'après Word2Vec

L'idée est d'utiliser la représentation de Word2Vec des mots pour faire du clustering sur tout les mots contenus dans la base. On se place ensuite au niveau de chaque tweet et on le représente par un vecteur qui compte le nombre de mots contenu dans chaque cluster. Plus pour analyser quels sont les mots qui se rapproche le plus selon la distance euclidienne ? (plutot que la cos similarité).

In [ ]:
# Représentation Word2Vec de chaque mot 

clean_text=clean_text_second(df.Tweet,threshold=1000)

print("Entrainement du modèle Word2Vec ...")
model = Word2Vec(clean_text, size=100, window=5, min_count=3, workers=4) 

model.train(clean_text, total_examples=len(clean_text), epochs=10) #réseau de neuronne du Word2Vec
model_wv = model.wv

In [ ]:
# Application du modèle W2v sur chaque token du corpus

clean_text = reduce(add, clean_text)
clean_text = set(clean_text)

def Representation_W2v(tokenCorpus_unique):
    new_sample = {}
    number_not_in_vocab=0
    for token in tokenCorpus_unique:
            try : 
                new_sample[token]=list(model_wv.get_vector(token))
            except: number_not_in_vocab=number_not_in_vocab+1
    
    print('{} mots sur {} ne sont pas dans le vocabulaire du modèle Word2Vec entrainé'.format(str(number_not_in_vocab),str(len(tokenCorpus_unique))))
    return new_sample

Representation_for_clustering=pd.DataFrame(Representation_W2v(clean_text)).T
Representation_for_clustering

In [ ]:
Representation_for_clustering

In [ ]:
# Application du clustering 

n_cluster = 5

kmeans = KMeans(n_clusters=n_cluster) 
kmeans.fit(Representation_for_clustering)
clusters=kmeans.predict(Representation_for_clustering).tolist()

data=[]
for x in range(len(clusters)):
    data.append([Representation_for_clustering.index.tolist()[x],clusters[x]])
    
    
results=pd.DataFrame(data= data , columns=['token','cluster'])

In [ ]:
for k in range(n_cluster):
    print('Nombre de mots dans le cluster numéro {} : '.format(k)+str(len(results[results.cluster==k])))
    print(' ')
    print('#'*20)
    print(' ')

In [ ]:
# Etude de chaque cluster spécifiquement
cluster=2

list_word=''
for word_index in range(20):
    word=list(results[results.cluster==cluster]['token'])[word_index]
    list_word=list_word+' , '+word
print('Parmis les mots du cluster {}, il y a : '.format(cluster)+list_word)

Interprétation des clusters ? Refaire avec des modèles pré-entrainés ? 

# Représentation de tweet avec des modèles pré-entrainés : Fast2vec et Bert

### Fast2vec pre-trained

### Bert pre-trained

In [71]:
#pip install pytorch-pretrained-bert

In [74]:
clean_text=clean_text_second(df.Tweet,threshold=1000)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Input formating for BERT

#tokenization spécifique BERT
def tokenization_BERT(tweet,tokenizer):
    text=''
    for word in tweet:
        text=text+' '+word
    tokenized_text='[CLS]'+text+' [SEP]'
    tokenized_text = tokenizer.tokenize(tokenized_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [0] * len(tokenized_text)
    
    return(indexed_tokens,segments_ids)

##########################################
##########################################

# Embedding du tweet
def BERT_embedding(tweet,tokenizer):
    
    #tokenization et convertion des inputs en tensors
    index, segments= tokenization_BERT(tweet,tokenizer)
    
    tokens_tensor = torch.tensor([index])
    segments_tensors = torch.tensor([segments])
    
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)

    token_embeddings = torch.stack(encoded_layers, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    
    token_vecs = encoded_layers[11][0]
    sentence_embedding = torch.mean(token_vecs, dim=0)
    
    return(list(sentence_embedding.numpy())) #return le vecteur du tweet
    





In [36]:
# Import du modèle

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

100%|██████████| 407873900/407873900 [13:53<00:00, 489066.62B/s] 


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [77]:
# Représentation des 5 premiers tweets de la base

for tweet_index in range(5):
    print('Tweet initial : '+df.iloc[tweet_index]['Tweet'])
    print(' ')
    text=''
    for word in clean_text[tweet_index]:
        text=text+' '+word
    print('Tweet nettoyé : '+text)
    print(' ')
    print('10 premières valeurs de la représentation vectorielle : '+str(BERT_embedding(clean_text[tweet_index],tokenizer)[0:10]))
    print(' ')
    print('#'*20)
    print(' ')

Tweet initial : Global warming report urges governments to act|BRUSSELS, Belgium (AP) - The world faces increased hunger and .. 
 
Tweet nettoyé :  global warming report urges governments to act brussels belgium ap the world faces increased hunger and
 
10 premières valeurs de la représentation vectorielle : [-0.20707315, -0.1527531, 0.31561086, -0.102150574, -0.0494032, -0.15978688, 0.12970391, 0.61809164, -0.43857074, -0.18857116]
 
####################
 
Tweet initial : Fighting poverty and global warming in Africa 
 
Tweet nettoyé :  fighting poverty and global warming in africa
 
10 premières valeurs de la représentation vectorielle : [-0.3177802, -0.19386752, -0.51339495, 0.08279662, 0.005352762, -0.3944389, 0.124147385, 0.6389174, -0.39283025, -0.5550124]
 
####################
 
Tweet initial : Carbon offsets: How a Vatican forest failed to reduce global warming 
 
Tweet nettoyé :  carbon offsets how a vatican forest failed to reduce global warming
 
10 premières valeurs de la 

In [79]:
# Représentation de la base entière par BERT

dataframe={}
for tweet_index in tqdm(range(len(clean_text))):
    dataframe[tweet_index]=BERT_embedding(clean_text[tweet_index],tokenizer)
df_BERT_embedding=pd.DataFrame(dataframe)

df_BERT_embedding

100%|██████████| 5539/5539 [14:40<00:00,  6.29it/s]


In [80]:
df_BERT_embedding

,0,1,2,3,4,5,6,7,8,9,...,5529,5530,5531,5532,5533,5534,5535,5536,5537,5538
0,-0.207073,-0.317780,-0.111854,-0.270964,-0.236785,0.393400,0.085322,-0.178012,-0.101252,-0.159416,...,0.012333,-0.060874,-0.126322,-0.021645,-0.281184,-0.400154,0.051658,-0.010287,-0.018927,0.059379
1,-0.152753,-0.193868,-0.214335,0.050122,0.089003,0.467884,-0.049446,-0.168504,-0.148704,0.046357,...,0.464100,0.068738,-0.085134,0.265985,0.009084,-0.256461,-0.054771,0.161429,0.158623,0.142215
2,0.315611,-0.513395,-0.171047,-0.220348,0.507813,0.210394,0.098238,0.339805,-0.073870,-0.197705,...,0.088426,0.205937,-0.083525,0.493496,0.063933,-0.183564,0.298856,0.030481,0.391081,0.182608
3,-0.102151,0.082797,0.315771,0.321847,0.069788,-0.083486,-0.287598,0.390049,-0.002181,0.123149,...,0.253761,0.301991,0.052356,-0.032803,-0.024530,0.033262,0.032705,-0.085691,0.183931,0.228363
4,-0.049403,0.005353,0.137571,0.305090,0.178619,-0.151924,0.420903,0.448748,0.250639,0.046807,...,0.190971,0.264646,0.189702,-0.009137,-0.083970,0.159400,0.214817,0.177870,0.028438,0.027777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,-0.185915,-0.301050,-0.400634,-0.265072,-0.098970,-0.075689,-0.036328,-0.013185,-0.134336,-0.092333,...,-0.330396,-0.570294,-0.547132,-0.065307,-0.157501,-0.436011,-0.042413,-0.412418,-0.248569,-0.000809
764,0.117077,0.167589,0.102872,-0.133903,0.263657,0.123565,-0.013714,-0.358873,-0.198908,-0.091171,...,-0.148618,0.399273,0.060670,0.016221,0.029190,-0.095360,-0.044953,-0.331940,0.103792,0.119760
765,-0.721387,-0.430743,-0.392623,-0.531327,-0.353686,-0.185703,-0.366322,-0.413659,-0.478867,-0.323900,...,-0.505612,-0.060666,0.036509,-0.288299,-0.223615,-0.296756,-0.381793,-0.071939,-0.125099,-0.096072
766,-0.034098,-0.192657,-0.079638,-0.279678,0.036645,0.247101,-0.196660,-0.090664,-0.095181,0.272903,...,-0.194793,0.411907,0.265059,0.317590,0.174986,0.257059,0.155324,0.001736,0.281513,0.467625
